<a href="https://colab.research.google.com/github/semthedev/vk-rustore-data-analytics-test-task/blob/main/2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
import pandas as pd
from datetime import datetime, timedelta

In [45]:
# Исходные данные - события пользователей в магазине приложений
# userId - идентификатор пользователя
# eventName - событие (регистрация, загрузка, покупка и т. д.)
# time - время события
# product - идентификатор продукта (если применимо)
data = [
    (1,  'register', '2023-03-06 10:00:00', None),
    (1,  'download', '2023-03-07 11:00:00', 'p1'),
    (2,  'register', '2023-03-06 12:00:00', None),
    (2,  'download', '2023-03-13 14:00:00', 'p2'),
    (3,  'register', '2023-03-08 08:30:00', None),
    (3,  'launch',   '2023-03-09 09:45:00', None),
    (4,  'register', '2023-03-13 15:00:00', None),
    (4,  'download', '2023-03-14 16:30:00', 'p3'),
    (5,  'register', '2023-03-13 17:00:00', None),
    (6,  'register', '2023-03-20 18:00:00', None),
    (6,  'download', '2023-03-21 19:00:00', 'p2'),
    (7,  'register', '2023-03-20 20:00:00', None),
    (8,  'pageVisit','2023-04-01 14:00:00', 'p4'),
    (9,  'buy',      '2023-04-02 15:00:00', 'p5'),
    (10, 'update',   '2023-04-03 16:00:00', 'p6'),
]

In [46]:
# Создаем DataFrame из списка данных
df = pd.DataFrame(data, columns=['userId', 'eventName', 'time', 'product'])

# Преобразуем столбец времени в формат datetime
df['time'] = pd.to_datetime(df['time'])

# Фильтруем только события регистрации и загрузки приложения
registrations = df[df['eventName'] == 'register']
downloads = df[df['eventName'] == 'download']

In [47]:
# Определяем дату первой регистрации для каждого пользователя
registration_dates = registrations.groupby('userId')['time'].min().reset_index()
registration_dates.rename(columns={'time': 'registered'}, inplace=True)

# Определяем дату первой загрузки для каждого пользователя
first_downloads = downloads.groupby('userId')['time'].min().reset_index()
first_downloads.rename(columns={'time': 'downloaded'}, inplace=True)

In [48]:
# Функция, которая возвращает начало недели (понедельник) для заданной даты
def get_monday_of_week(date):
    return date.to_period('W').start_time

# Добавляем столбцы с неделями регистрации и загрузки
registration_dates['reg_week'] = registration_dates['registered'].apply(get_monday_of_week)
first_downloads['download_week'] = first_downloads['downloaded'].apply(get_monday_of_week)

In [49]:
# Объединяем данные о регистрации и первой загрузке по userId
cohort = pd.merge(registration_dates, first_downloads, on='userId', how='left')

# Считаем количество пользователей, зарегистрировавшихся в каждую неделю
weekly_users = cohort.groupby('reg_week').size().reset_index(name='users')

# Подсчитываем, сколько пользователей загрузили приложение в ту же неделю, когда зарегистрировались
same_week_downloads = cohort[cohort['reg_week'] == cohort['download_week']].groupby('reg_week').size().reset_index(name='same_week_downloads')

# Объединяем данные о количестве зарегистрированных пользователей и загрузок в ту же неделю
result = pd.merge(weekly_users, same_week_downloads, on='reg_week', how='left')

# Заполняем пропущенные значения нулями (если в какой-то неделе не было загрузок)
result['same_week_downloads'].fillna(0, inplace=True)

# Рассчитываем коэффициент конверсии (CR) как процент пользователей, скачавших приложение в неделю регистрации
result['cr'] = (result['same_week_downloads'] / result['users'] * 100).round(2)

# Переименовываем колонку reg_week → week для итогового результата
result.rename(columns={'reg_week': 'week'}, inplace=True)

# Выводим итоговую таблицу, отсортированную по неделям
print(result[['week', 'users', 'cr']].sort_values(by='week', ascending=True))

        week  users     cr
0 2023-03-06      3  33.33
1 2023-03-13      2  50.00
2 2023-03-20      2  50.00


<ipython-input-49-281d72938e27>:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  result['same_week_downloads'].fillna(0, inplace=True)
